In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern, WhiteKernel

from sklearn.decomposition import PCA

from scipy.stats import zscore

np.random.seed(1)

combo = pd.read_csv('2016-2017_combo.csv')

df = pd.read_excel('cts.xlsx', sheet_name='2016-2017')
to_keep = ['Team Name', 'Adj Off Efficiency', 'FG%', 'Avg. Scoring margin',
           'Rebounds', 'Wins Last 10 Games ', 'Turnovers per game ']
df = df[to_keep]
# numeric_cols = df.select_dtypes(include=[np.number]).columns
# df[numeric_cols] = df[numeric_cols].apply(zscore)

In [2]:
Xs = []
ys = []

for idx, row in combo.iterrows():
    winner = row['Winner']
    loser = row['Loser']

    w_v = np.array(df[df['Team Name'] == winner])[0][1:]
    l_v = np.array(df[df['Team Name'] == loser])[0][1:]

    Xs.append(np.array([w_v, l_v]).flatten())
    ys.append(np.array([row['Winner Points'] - row['Loser points']]))

X = np.array(Xs)
y = np.array(ys)

coval = pd.read_csv('2015-2016_combo.csv')
val = pd.read_excel('cts.xlsx', sheet_name='2015-2016')
val = val[to_keep]
# numeric_cols = val.select_dtypes(include=[np.number]).columns
# val[numeric_cols] = val[numeric_cols].apply(zscore)

Xvs = []
yvs = []

for idx, row in coval.iterrows():
    winner = row['Winner']
    loser = row['Loser']

    try:
        w_v = np.array(val[val['Team Name'] == winner])[0][1:]
        l_v = np.array(val[val['Team Name'] == loser])[0][1:]
        Xvs.append(np.array([w_v, l_v]).flatten())
        yvs.append(np.array([row['Winner Points'] - row['Loser points']]))
    except:
        print(winner)
        print(loser)

Xv = np.array(Xvs)
yv = np.array(yvs)

Syracuse
MTSU
MTSU
Michigan State


In [3]:
rep = -1000
count = 0
rbest = 0
obest = 0
print('starting')
for rbf_val in np.linspace(0.001, 100, 20):
    print(f'{count}%')
    count += 1
    for outside in np.linspace(.001, 100, 20):
        gp = GaussianProcessRegressor(outside * RBF(rbf_val),
                                      n_restarts_optimizer=10,
                                      normalize_y=True)
        gp.fit(X, y)
        score = gp.score(Xv, yv)
        if score > rep:
            rbest = rbf_val
            obest = outside
            rep = score

print('GP Validation R^2: ' + str(rep))
print(f'Best RBF Length Scale: {rbest}')
print(f'Best RBF Kernel Scale: {obest}')

starting
0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
GP Validation R^2: -0.021056828457099552
Best RBF Length Scale: 0.001
Best RBF Kernel Scale: 0.001


In [4]:
rep = -1000
wbest = 0
for wkv in np.linspace(0.0000001, 100, 100):
    kernel = obest * RBF(rbest) + WhiteKernel(wkv)
    
    gp = GaussianProcessRegressor(kernel,
                                  n_restarts_optimizer=10,
                                  normalize_y=True)
    gp.fit(X, y)
    score = gp.score(Xv, yv)
    if score > rep:
        wbest = wkv
        rep = score
        
print('GP Validation R^2: ' + str(rep))
print(f'Best White Noise Parameter: {wbest}')

GP Validation R^2: -0.017788708256136765
Best White Noise Parameter: 69.69696972727272


In [5]:
# rbest = 63
# obest = 26
# wbest = 34

kernel = obest * RBF(rbest) + WhiteKernel(wbest)
gp = GaussianProcessRegressor(kernel,
                                  n_restarts_optimizer=10,
                                  normalize_y=True)
gp.fit(X, y)
rep = gp.score(Xv, yv)

print('GP Validation R^2: ' + str(rep))

GP Validation R^2: -0.021056828457099552
